Our project uses two datasets:
1. A [Yelp](https://www.yelp.com/dataset) dataset: The Yelp dataset covers 160,585 businesses across 8 metropolitan areas. There are a couple of different tables: business, review, user, checkin, tip, and photo. We’ll probably focus a lot of our attention on the business and review tables. (--> nothing in NYC https://github.com/Yelp/dataset-examples/issues/47)

2. An [Airbnb](http://insideairbnb.com/get-the-data.html) dataset: The AIrBnB dataset, provided by ‘Inside Airbnb’, covers 36,923 locations across the entire NYC metropolitan area. Important columns that we have are latitude and longitude, number of bedrooms, as well as review statistics for a location etc. 


To prepare the datasets for ingestion into the database, we want to:
1. Clean missing / misentered values
2. Detect and solve entity resolution problems
3. Replace categorical variables with numeric indicators for efficiency
4. Export the data

## Cleaning of Data for Stay cozy with good eats

we will look to : 
*   Removal of rows with missing values
*   Replacement of missing values with default values
*   Dectection of entity resolution problems
*   Simple entity resolution
*   Removal of unpaired entities
*   Replacement of categorical variables with indicators
*   Identification of candidate indices
*   Data exportation

In [101]:
import numpy as np
import pandas as pd
from google.colab import drive
import math
import requests 
import json

In [102]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)
#success code: 4/1AX4XfWg77OvWTHG6cqauPcN_8-a3FWdeeWTGs1yBFShQYlfue6DcdgYd-70

Mounted at /content/drive


In [103]:
yelp_path = '/content/drive/Shareddrives/CIS550/raw_datasets/yelp.csv' # we dont use this 
airbnb_path = '/content/drive/Shareddrives/CIS550/raw_datasets/airbnb.csv'

In [104]:
yelp_raw = pd.read_csv(yelp_path) # we dont use this 
airbnb_raw = pd.read_csv(airbnb_path)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,7,17,29,49,60,62,79,86,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Airbnb Data Cleanup
For the Airbnb Data, we want to remove a lot of columns and only keep a few. We also want to have a host table. 

We will also look for missing or misentered values, create categorical values, and check for entity resolution. We also only want New York Data.

In [105]:
airbnb_raw.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2265,https://www.airbnb.com/rooms/2265,20210914193120,2021-09-15,Zen-East in the Heart of Austin (monthly rental),Zen East is situated in a vibrant & diverse mu...,NaN,https://a0.muscache.com/pictures/4187/52d4f5d0...,2466,https://www.airbnb.com/users/show/2466,Paddy,2008-08-23,"Austin, Texas, United States",I am a long time resident of Austin. I earned ...,within a few hours,100%,75%,t,https://a0.muscache.com/im/users/2466/profile_...,https://a0.muscache.com/im/users/2466/profile_...,East Downtown,3.0,3.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,NaN,78702,NaN,30.27752,-97.71377,Entire residential home,Entire home/apt,4,NaN,2 baths,2.0,2.0,"[""Free street parking"", ""Free dryer \u2013 In ...",$200.00,7,180,7.0,7.0,180.0,180.0,7.0,180.0,NaN,t,0,7,37,37,2021-09-15,26,2,0,2015-11-09,2021-07-02,4.68,4.74,4.83,4.77,4.83,4.26,4.35,NaN,f,3,2,1,0,0.36
1,5245,https://www.airbnb.com/rooms/5245,20210914193120,2021-09-15,"Eco friendly, Colorful, Clean, Cozy monthly share",Situated in a vibrant & diverse multicultural ...,NaN,https://a0.muscache.com/pictures/5167505/b33b5...,2466,https://www.airbnb.com/users/show/2466,Paddy,2008-08-23,"Austin, Texas, United States",I am a long time resident of Austin. I earned ...,within a few hours,100%,75%,t,https://a0.muscache.com/im/users/2466/profile_...,https://a0.muscache.com/im/users/2466/profile_...,East Downtown,3.0,3.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,NaN,78702,NaN,30.27614,-97.71320,Private room in residential home,Private room,2,NaN,1 private bath,1.0,2.0,"[""Free street parking"", ""Air conditioning"", ""L...",$114.00,30,90,30.0,30.0,90.0,90.0,30.0,90.0,NaN,t,0,28,58,58,2021-09-15,9,0,0,2018-03-14,2017-02-24,4.57,4.80,4.20,5.00,4.40,4.75,4.50,NaN,f,3,2,1,0,0.21
2,5456,https://www.airbnb.com/rooms/5456,20210914193120,2021-09-15,"Walk to 6th, Rainey St and Convention Ctr",Great central location for walking to Convent...,My neighborhood is ideally located if you want...,https://a0.muscache.com/pictures/14084884/b5a3...,8028,https://www.airbnb.com/users/show/8028,Sylvia,2009-02-16,"Austin, Texas, United States",I am a licensed Real Estate Broker and owner o...,within an hour,100%,92%,t,https://a0.muscache.com/im/users/8028/profile_...,https://a0.muscache.com/im/users/8028/profile_...,East Downtown,1.0,1.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Austin, Texas, United States",78702,NaN,30.26057,-97.73441,Entire guesthouse,Entire home/apt,3,NaN,1 bath,1.0,2.0,"[""Free street parking"", ""Patio or balcony"", ""E...",$104.00,2,90,2.0,2.0,90.0,90.0,2.0,90.0,NaN,t,17,35,55,324,2021-09-15,570,35,2,2019-11-01,2021-09-01,4.83,4.87,4.85,4.89,4.80,4.73,4.78,NaN,f,1,1,0,0,24

### Colums that the Airbnb csv started with : 

```
Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'neighbourhood_cleansed',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object')
```

### Our DDL: 
```
CREATE TABLE Airbnb(
Id int PRIMARY KEY,
Listing_url varchar(20),
Latitude float,
Longitude float,
bathrooms float, 
bedrooms int,
price float, 
review_scores_value float, 
review_scores_count int, 
neighborhood varchar(40))
foreign key host_id references Host.host_id

CREATE TABLE Host(
host_id int PRIMARY KEY,
...

```


### Columns we would like to thus keep: 
```
Airbnb: [
'id', 'listing_url', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location', 'instant_bookable']
# id is the primary key
# host_id is a foreign key to Host

Host: [
  'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost',
       'host_neighbourhood', 
       'host_total_listings_count'
]
# host_id is the primary key
```

In [106]:
airbnb = airbnb_raw[[
'id', 'listing_url', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location', 'instant_bookable', 'neighbourhood_cleansed']]
host_raw = airbnb_raw[[
  'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost',
       'host_neighbourhood', 
       'host_total_listings_count'
]]
airbnb.head()

,id,listing_url,name,description,neighborhood_overview,picture_url,host_id,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,instant_bookable,neighbourhood_cleansed
0,2265,https://www.airbnb.com/rooms/2265,Zen-East in the Heart of Austin (monthly rental),Zen East is situated in a vibrant & diverse mu...,NaN,https://a0.muscache.com/pictures/4187/52d4f5d0...,2466,30.27752,-97.71377,Entire residential home,Entire home/apt,4,2 baths,2.0,2.0,"[""Free street parking"", ""Free dryer \u2013 In ...",$200.00,7,180,26,4.68,4.83,4.77,4.83,4.26,f,78702
1,5245,https://www.airbnb.com/rooms/5245,"Eco friendly, Colorful, Clean, Cozy monthly share",Situated in a vibrant & diverse multicultural ...,NaN,https://a0.muscache.com/pictures/5167505/b33b5...,2466,30.27614,-97.71320,Private room in residential home,Private room,2,1 private bath,1.0,2.0,"[""Free street parking"", ""Air conditioning"", ""L...",$114.00,30,90,9,4.57,4.20,5.00,4.40,4.75,f,78702
2,5456,https://www.airbnb.com/rooms/5456,"Walk to 6th, Rainey St and Convention Ctr",Great central location for walking to Convent...,My neighborhood is ideally located if you want...,https://a0.muscache.com/pictures/14084884/b5a3...,8028,30.26057,-97.73441,Entire guesthouse,Entire home/apt,3,1 bath,1.0,2.0,"[""Free street parking"", ""Patio or balcony"", ""E...",$104.00,2,90,570,4.83,4.85,4.89,4.80,4.73,f,78702
3,5769,https://www.airbnb.com/rooms/5769,NW Austin Room,<b>The space</b><br />Looking for a comfortabl...,Quiet neighborhood with lots of trees and good...,https://a0.muscache.com/pictures/23822033/ac94...,8186,30.45697,-97.78422,Private room in residential home,Private room,2,1 shared bath,1.0,1.0,"[""Hair dryer"", ""First aid kit"", ""Dishes and si...",$39.00,1,14,264,4.90,4.86,4.91,4.94,4.74,f,78729
4,6413,https://www.airbnb.com/rooms/6413,Gem of a Studio near Downtown,"Great studio apartment, perfect a single perso...",Travis Heights is one of the oldest neighborho...,https://a0.muscache.com/pictures/miso/Hosting-...,13879,30.24885,-97.73587,Entire guesthouse,Entire home/apt,2,1 bath,NaN,1.0,"[""Free street parking"", ""Patio or balcony"", ""A...",$109.00,3,365,117,4.97,4.99,4.99,4.98,4.86,t,78704


#Cleaning Airbnb

here we will remove nulls from the columns where we deem them not appropriate. 

We will also rename 2 columns to 'neighbourhood' and 'borough'

We will then change data types

In [107]:
airbnb.head()

,id,listing_url,name,description,neighborhood_overview,picture_url,host_id,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,instant_bookable,neighbourhood_cleansed
0,2265,https://www.airbnb.com/rooms/2265,Zen-East in the Heart of Austin (monthly rental),Zen East is situated in a vibrant & diverse mu...,NaN,https://a0.muscache.com/pictures/4187/52d4f5d0...,2466,30.27752,-97.71377,Entire residential home,Entire home/apt,4,2 baths,2.0,2.0,"[""Free street parking"", ""Free dryer \u2013 In ...",$200.00,7,180,26,4.68,4.83,4.77,4.83,4.26,f,78702
1,5245,https://www.airbnb.com/rooms/5245,"Eco friendly, Colorful, Clean, Cozy monthly share",Situated in a vibrant & diverse multicultural ...,NaN,https://a0.muscache.com/pictures/5167505/b33b5...,2466,30.27614,-97.71320,Private room in residential home,Private room,2,1 private bath,1.0,2.0,"[""Free street parking"", ""Air conditioning"", ""L...",$114.00,30,90,9,4.57,4.20,5.00,4.40,4.75,f,78702
2,5456,https://www.airbnb.com/rooms/5456,"Walk to 6th, Rainey St and Convention Ctr",Great central location for walking to Convent...,My neighborhood is ideally located if you want...,https://a0.muscache.com/pictures/14084884/b5a3...,8028,30.26057,-97.73441,Entire guesthouse,Entire home/apt,3,1 bath,1.0,2.0,"[""Free street parking"", ""Patio or balcony"", ""E...",$104.00,2,90,570,4.83,4.85,4.89,4.80,4.73,f,78702
3,5769,https://www.airbnb.com/rooms/5769,NW Austin Room,<b>The space</b><br />Looking for a comfortabl...,Quiet neighborhood with lots of trees and good...,https://a0.muscache.com/pictures/23822033/ac94...,8186,30.45697,-97.78422,Private room in residential home,Private room,2,1 shared bath,1.0,1.0,"[""Hair dryer"", ""First aid kit"", ""Dishes and si...",$39.00,1,14,264,4.90,4.86,4.91,4.94,4.74,f,78729
4,6413,https://www.airbnb.com/rooms/6413,Gem of a Studio near Downtown,"Great studio apartment, perfect a single perso...",Travis Heights is one of the oldest neighborho...,https://a0.muscache.com/pictures/miso/Hosting-...,13879,30.24885,-97.73587,Entire guesthouse,Entire home/apt,2,1 bath,NaN,1.0,"[""Free street parking"", ""Patio or balcony"", ""A...",$109.00,3,365,117,4.97,4.99,4.99,4.98,4.86,t,78704


In [108]:
#rename columns 'neighbourhood_cleansed', 'neighbourhood_group_cleansed'
airbnb = airbnb.rename(columns={'bathrooms_text': 'bathrooms_details', 'neighbourhood_cleansed': 'postal_code'})
airbnb.head()

,id,listing_url,name,description,neighborhood_overview,picture_url,host_id,latitude,longitude,property_type,room_type,accommodates,bathrooms_details,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,instant_bookable,postal_code
0,2265,https://www.airbnb.com/rooms/2265,Zen-East in the Heart of Austin (monthly rental),Zen East is situated in a vibrant & diverse mu...,NaN,https://a0.muscache.com/pictures/4187/52d4f5d0...,2466,30.27752,-97.71377,Entire residential home,Entire home/apt,4,2 baths,2.0,2.0,"[""Free street parking"", ""Free dryer \u2013 In ...",$200.00,7,180,26,4.68,4.83,4.77,4.83,4.26,f,78702
1,5245,https://www.airbnb.com/rooms/5245,"Eco friendly, Colorful, Clean, Cozy monthly share",Situated in a vibrant & diverse multicultural ...,NaN,https://a0.muscache.com/pictures/5167505/b33b5...,2466,30.27614,-97.71320,Private room in residential home,Private room,2,1 private bath,1.0,2.0,"[""Free street parking"", ""Air conditioning"", ""L...",$114.00,30,90,9,4.57,4.20,5.00,4.40,4.75,f,78702
2,5456,https://www.airbnb.com/rooms/5456,"Walk to 6th, Rainey St and Convention Ctr",Great central location for walking to Convent...,My neighborhood is ideally located if you want...,https://a0.muscache.com/pictures/14084884/b5a3...,8028,30.26057,-97.73441,Entire guesthouse,Entire home/apt,3,1 bath,1.0,2.0,"[""Free street parking"", ""Patio or balcony"", ""E...",$104.00,2,90,570,4.83,4.85,4.89,4.80,4.73,f,78702
3,5769,https://www.airbnb.com/rooms/5769,NW Austin Room,<b>The space</b><br />Looking for a comfortabl...,Quiet neighborhood with lots of trees and good...,https://a0.muscache.com/pictures/23822033/ac94...,8186,30.45697,-97.78422,Private room in residential home,Private room,2,1 shared bath,1.0,1.0,"[""Hair dryer"", ""First aid kit"", ""Dishes and si...",$39.00,1,14,264,4.90,4.86,4.91,4.94,4.74,f,78729
4,6413,https://www.airbnb.com/rooms/6413,Gem of a Studio near Downtown,"Great studio apartment, perfect a single perso...",Travis Heights is one of the oldest neighborho...,https://a0.muscache.com/pictures/miso/Hosting-...,13879,30.24885,-97.73587,Entire guesthouse,Entire home/apt,2,1 bath,NaN,1.0,"[""Free street parking"", ""Patio or balcony"", ""A...",$109.00,3,365,117,4.97,4.99,4.99,4.98,4.86,t,78704


In [109]:
set(airbnb['postal_code'])

{78701,
 78702,
 78703,
 78704,
 78705,
 78712,
 78717,
 78719,
 78721,
 78722,
 78723,
 78724,
 78725,
 78726,
 78727,
 78728,
 78729,
 78730,
 78731,
 78732,
 78733,
 78734,
 78735,
 78736,
 78737,
 78738,
 78739,
 78741,
 78742,
 78744,
 78745,
 78746,
 78747,
 78748,
 78749,
 78750,
 78751,
 78752,
 78753,
 78754,
 78756,
 78757,
 78758,
 78759}

In [110]:
list_non_null_airbnb_fields = ['id', 'listing_url', 'name', 'picture_url', 'host_id', 'latitude',
'longitude', 'property_type', 'room_type', 'accommodates',
'bathrooms_details', 'bedrooms', 'beds', 'amenities', 'price',
'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating',
'review_scores_cleanliness', 'review_scores_checkin',
'review_scores_communication', 'review_scores_location', 'instant_bookable', 'postal_code']
missing_airbnb = airbnb.isnull()
missing_airbnb = missing_airbnb[list_non_null_airbnb_fields].any(axis=1)
airbnb.size #281034
airbnb = airbnb.loc[(~missing_airbnb).values, :]
airbnb.size #201069

201069

In [111]:
set(airbnb['bedrooms'])

{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 12.0, 13.0, 15.0, 23.0}

In [112]:
# making a dict to convert bathroom input to ints 
bathroom_dict = {'0 baths': 0,
 '0 shared baths': 0,
 '1 bath' : 1,
 '1 private bath': 1,
 '1 shared bath': 1,
 '1.5 baths': 1.5,
 '1.5 shared baths': 1.5,
 '10.5 baths': 10,
 '12 baths': 12,
 '17 baths': 17,
 '15.5 baths': 15.5,
 '2 baths': 2,
 '2 shared baths': 2,
 '2.5 baths': 2.5,
 '2.5 shared baths': 2.5,
 '3 baths': 3,
 '3 shared baths': 3,
 '3.5 baths': 3.5,
 '3.5 shared baths': 3.5,
 '4 baths': 4,
 '4 shared baths': 4,
 '4.5 baths': 4.5,
 '5 baths': 5,
 '5.5 baths': 5.5,
 '6 baths': 6,
 '6 shared baths': 6,
 '6.5 baths': 6.5,
 '7 baths': 7,
 '7.5 baths': 7.5,
 '8 baths': 8,
 '8.5 baths': 8.5,
 'Half-bath': 0.5,
 '9 baths': 9,
 'Private half-bath': 0.5,
 'Shared half-bath': 0.5}

In [113]:
# convert to floats
#'id', 'host_id', 'amenities', 'price', 'instant_bookable'
convert_to_float = ['review_scores_rating',
'review_scores_cleanliness', 'review_scores_checkin',
'review_scores_communication', 'latitude',
'longitude', 'review_scores_location']
convert_to_int = ['minimum_nights', 'maximum_nights', 'number_of_reviews', 'bedrooms', 'beds', 'accommodates', 'postal_code']
convert_to_str = ['listing_url', 'name', 'description',
'neighborhood_overview', 'picture_url',  'property_type', 'bathrooms_details']

for col in convert_to_float:  
  airbnb[col] = airbnb[col].apply(lambda x : float(x))
for col in convert_to_int: 
  airbnb[col] = airbnb[col].apply(lambda x : int(x))
for col in convert_to_str:  
  airbnb[col] = airbnb[col].apply(lambda x : str(x))

# here we are converting the prices to floats without the '$'
airbnb['price'] = airbnb['price'].apply(lambda x : float(x[1:].replace(',','')))

# here we are converting the bathrooms_text to just an int
airbnb['bathrooms'] = airbnb['bathrooms_details'].apply(lambda x : bathroom_dict[x])

# 'room_type' is already an enum :
# {'Entire home/apt', 'Hotel room', 'Private room', 'Shared room'}
set(airbnb['room_type'])

# make 'instant_bookable' a boolean
airbnb['instant_bookable'] = airbnb['instant_bookable'].apply(lambda x : False if x == 'f' else True)

In [114]:
set(host['host_response_time'])

{'a few days or more',
 nan,
 'within a day',
 'within a few hours',
 'within an hour'}

In [115]:
len(set(host['host_id'])-set(airbnb['host_id'])), len(set(airbnb['host_id'])-set(host['host_id'])), len(set(host['host_id'])), len(set(airbnb['host_id']))

(0, 0, 5068, 5068)

In [116]:
airbnb.dtypes

id                               int64
listing_url                     object
name                            object
description                     object
neighborhood_overview           object
picture_url                     object
host_id                          int64
latitude                       float64
longitude                      float64
property_type                   object
room_type                       object
accommodates                     int64
bathrooms_details               object
bedrooms                         int64
beds                             int64
amenities                       object
price                          float64
minimum_nights                   int64
maximum_nights                   int64
number_of_reviews                int64
review_scores_rating           float64
review_scores_cleanliness      float64
review_scores_checkin          float64
review_scores_communication    float64
review_scores_location         float64
instant_bookable         

In [117]:
len(airbnb.index), len(set(airbnb.id))

(7447, 7447)

#Cleaning Host

In [118]:
len(host_raw.index), len(set(host_raw.host_id))

(10809, 7008)

In [119]:
host = pd.DataFrame(columns=[
  'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost',
       'host_neighbourhood', 
       'host_total_listings_count'])
for col, row in airbnb.iterrows():
  new_host = host_raw.loc[host_raw['host_id'] == row['host_id']]
  #if not (host['host_id'] == row['host_id']).any():
  if row['host_id'] not in host.values:
    host = host.append(new_host)
len(set(host.index)), len(set(host.host_id))
host.head()

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_total_listings_count
0,2466,https://www.airbnb.com/users/show/2466,Paddy,2008-08-23,"Austin, Texas, United States",I am a long time resident of Austin. I earned ...,within a few hours,100%,75%,t,East Downtown,3.0
1,2466,https://www.airbnb.com/users/show/2466,Paddy,2008-08-23,"Austin, Texas, United States",I am a long time resident of Austin. I earned ...,within a few hours,100%,75%,t,East Downtown,3.0
2886,2466,https://www.airbnb.com/users/show/2466,Paddy,2008-08-23,"Austin, Texas, United States",I am a long time resident of Austin. I earned ...,within a few hours,100%,75%,t,East Downtown,3.0
2,8028,https://www.airbnb.com/users/show/8028,Sylvia,2009-02-16,"Austin, Texas, United States",I am a licensed Real Estate Broker and owner o...,within an hour,100%,92%,t,East Downtown,1.0
3,8186,https://www.airbnb.com/users/show/8186,Elizabeth,2009-02-19,"Austin, Texas, United States",We're easygoing professionals that enjoy meeti...,within a few hours,100%,100%,t,SW Williamson Co.,1.0


In [120]:
len(set(host.index)), len(set(host.host_id))
# w diff if : (8349, 5068)
# w if of in (8349, 5068)
#dupes = host[host.duplicated(subset=['host_id'])]
host = host.drop_duplicates(subset=['host_id'])
len(set(host.index)), len(set(host.host_id))

(5068, 5068)

In [121]:
list_non_null_host_fields = ['host_id']
missing_hosts = host.isnull()
missing_hosts = missing_hosts[list_non_null_host_fields].any(axis=1)
host_raw = host.loc[(~missing_hosts).values, :]
len(set(host['host_id'])), len(host.index)

(5068, 5068)

In [122]:
host['host_id'].isnull().any().any()

False

In [128]:
host['host_response_rate']

0        100%
2        100%
3        100%
5        100%
6        100%
         ... 
10622    100%
10661    100%
10664    100%
10700    100%
10706     NaN
Name: host_response_rate, Length: 5068, dtype: object

We now want to determine what types we want for each column. Below we convert to ints and strings. We also convert the percent entries into ints. We also convert 'host_is_superhost' into a boolean. Additionally we verify that 'host_response_time' is prepped to become an enum of type {'a few days or more', 'within a day', 'within a few hours', 'within an hour'}


In [130]:
def perc_to_int(x): 
  return int(x[:-1])
#int(host.loc[500]['host_response_rate'][:-1])

# here we are converting the percentages to ints without the '%'
#host['host_response_rate'] = host['host_response_rate'].apply(lambda x : int(x[:-1]))
#host['host_response_rate'] = host['host_response_rate'].apply(lambda x: x)
#host['host_acceptance_rate'] = host['host_acceptance_rate'].apply(lambda x : int(x[:-1]))

# here we are converting the number of listings to ints
#host['host_total_listings_count'] = host['host_total_listings_count'].apply(lambda x : int(x))

# here we will convert to strings 
convert_to_str = ['host_url','host_name','host_location','host_about','host_neighbourhood']
for col in convert_to_str:  
  host[col] = host[col].apply(lambda x : str(x))

#host['host_url'] = host['host_url'].apply(lambda x : str(x))
#host['host_name'] = host['host_name'].apply(lambda x : str(x))
#host['host_location'] = host['host_location'].apply(lambda x : str(x))
#host['host_about'] = host['host_about'].apply(lambda x : str(x))
#host['host_neighbourhood'] = host['host_neighbourhood'].apply(lambda x : str(x))

# 'Host_response_time' is already an enum :
# host['host_response_time'] = host['host_response_time'].apply(lambda x : 'unknown' if x.isnull() else x)
set(host['host_response_time'])


{'a few days or more',
 nan,
 'within a day',
 'within a few hours',
 'within an hour'}

In [131]:
# make 'host_is_superhost' a boolean
host['host_is_superhost'] = host['host_is_superhost'].apply(lambda x : False if x == 'f' else True)

In [132]:
host.head()

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_total_listings_count
0,2466,https://www.airbnb.com/users/show/2466,Paddy,2008-08-23,"Austin, Texas, United States",I am a long time resident of Austin. I earned ...,within a few hours,100%,75%,True,East Downtown,3.0
2,8028,https://www.airbnb.com/users/show/8028,Sylvia,2009-02-16,"Austin, Texas, United States",I am a licensed Real Estate Broker and owner o...,within an hour,100%,92%,True,East Downtown,1.0
3,8186,https://www.airbnb.com/users/show/8186,Elizabeth,2009-02-19,"Austin, Texas, United States",We're easygoing professionals that enjoy meeti...,within a few hours,100%,100%,True,SW Williamson Co.,1.0
5,14156,https://www.airbnb.com/users/show/14156,Amy,2009-04-20,"Austin, Texas, United States","We are a family of four (with teenagers, all o...",within an hour,100%,97%,True,Zilker,2.0
6,25298,https://www.airbnb.com/users/show/25298,Karen,2009-07-11,"Austin, Texas, United States",I handle the reservations at the studio on the...,within an hour,100%,67%,False,East Riverside,1.0


We finish with the following for **airbnb**
```
id                               int64
listing_url                     object
name                            object
description                     object
neighborhood_overview           object
picture_url                     object
host_id                          int64
latitude                       float64
longitude                      float64
property_type                   object
room_type                       object
accommodates                     int64
bathrooms_details               object
bedrooms                         int64
beds                             int64
amenities                       object
price                          float64
minimum_nights                   int64
maximum_nights                   int64
number_of_reviews                int64
review_scores_rating           float64
review_scores_cleanliness      float64
review_scores_checkin          float64
review_scores_communication    float64
review_scores_location         float64
instant_bookable                  bool
postal_code                      int64
bathrooms                      float64
```
We finish with the following for **host**


```
host_id                       int64 PRIMARY KEY
host_url                     object
host_name                    object
host_since                   object
host_location                object
host_about                   object CAN BE NULL
host_response_time           object ENUM
host_response_rate            int64
host_acceptance_rate          int64
host_is_superhost              bool
host_neighbourhood           object
host_total_listings_count     int64

```




In [133]:
print(len(airbnb['id']))

7447


## Export Data


In [134]:
host.to_csv("host_table.csv", index=False)
airbnb.to_csv("airbnb_table.csv", index=True)

Finally, we download these files to our local machine, so we can put them into MySQL Workbench later. 

In [135]:
from google.colab import files
files.download('host_table.csv') 
files.download("airbnb_table.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Yelp Data Cleanup
We want to keep all of the columns so we will just look for missing or misentered values, create categorical values, and check for entity resolution. We also only want New York Data.

In [ ]:
yelp_raw.head()

In [ ]:
yelp = yelp_raw[yelp_raw['state']=='NY']
set(yelp_raw['state'])

In [ ]:
yelp.columns

## Important
you can see there is unfortunately no NYC info in the yelp dataset. The following is thus my attempt to grab it directly from the yelp API. It is inspired slightly by the code at the [following link](https://rspiro9.github.io/nyc_restaurant_yelp_and_inspection_analysis)

In [ ]:
pip install requests

In [ ]:
#set(yelp_raw['categories'])

connect to my API key:

In [ ]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)
				
# Pull in keys and specifically draw out the api key. I have removed the specific path to the keys 
# for security purposes:
#keys = get_keys("/Users/3a0K2dom2OuZm69n7rZMR78to5RL7OQgJWqgqIVnKXbFFcPQ4KgMoNWNPrDZ-DVlVUnGB4YuYOob8xU-Fm1WPaK4vLmbaLQ_ljszj7qUPyXNavWHRp-5OgfnPb6OYXYx/yelp_api.json")
api_key = '3a0K2dom2OuZm69n7rZMR78to5RL7OQgJWqgqIVnKXbFFcPQ4KgMoNWNPrDZ-DVlVUnGB4YuYOob8xU-Fm1WPaK4vLmbaLQ_ljszj7qUPyXNavWHRp-5OgfnPb6OYXYx'

# URL to pull data from:
url = 'https://api.yelp.com/v3/businesses/search'

# Identify headers:
headers = {'Authorization': 'Bearer {}'.format(api_key)}

In [ ]:
url = 'https://api.yelp.com/v3/businesses/search'
term = "Restaurants"
location =  'Manhattan, NY'
categories = "(restaurants, All)"
url_params = {
                        'location': location,
                        'term' : term,
                        'categories': categories,
                        'limit': 50,
                    }
response = requests.get(url, headers=headers, params=url_params)
response_json = response.json()

In [ ]:
#create empty dataframe for dataset
column_names = ['business_id', 'categories', 'name',
'postal_code', 'latitude', 'longitude', 'stars',
'review_count', 'price_range', 'hours', 'address', 'neighborhood', 'url']
restaurant =  pd.DataFrame(columns = column_names)

location =  'Manhattan, NY'
categories = "(restaurants, All)"

#looping through different offsets to get restaurants, then I will add to the dataframe
for offset in range(0, 1000, 50):
  search_limit = 50
  term = "Restaurants"
  url_params = {
                  'location': location,
                  'term' : term,
                  'limit': search_limit,
                  'categories': categories,
                  'offset': offset,
              }
  response = requests.get(url, headers=headers, params=url_params)
  if response.status_code == 400:
            print(f'400 Bad Request on offset {offset}')
            break
  if response.status_code == 500:
            print(f'500 Bad Request on offset {offset}')
            break
  response_json = response.json()
  print(response.status_code)
  for i in range(50):
    this_business = response_json['businesses'][i]
    try:
      price = this_business['price'] 
    except:
      price = 'UNKNOWN'
    #price = this_business['price'] if this_business['price'] else 'UNKNOWN'
    new_row = {'business_id': this_business['id'], 
               'categories':this_business['categories'][0]['alias'],
               'name':this_business['name'],
               'postal_code':this_business['location']['zip_code'],
               'latitude': this_business['coordinates']['latitude'],
               'longitude':this_business['coordinates']['longitude'],
               'stars':this_business['rating'],
               'review_count': this_business['review_count'],
               'price_range': price, #'hours': ,
               'address': this_business['location']['address1'], #'neighborhood': , 
               'url': this_business['url']}
    restaurant = restaurant.append(new_row, ignore_index=True)
  #print(response_json['businesses'][0]['name'], response_json, response)
  #print(response)

200
200
200
200
500 Bad Request on offset 200


In [ ]:
restaurant.head()

In [ ]:
#create empty dataframe for dataset
column_names = ['review_id',
'stars',
'date',
'useful',
'funny',
'cool',
'review_of']
review =  pd.DataFrame(columns = column_names)

url = 'https://api.yelp.com/v3/businesses/{id}/reviews'
#loop through restaurants in df
for restaurant_id in restaurant['business_id']: 
  search_limit = 50
  url = 'https://api.yelp.com/v3/businesses/' + restaurant_id + '/reviews'
  url_params = {
                'limit': search_limit,
              }
  response = requests.get(url, headers=headers, params=url_params)
  if response.status_code == 400:
            print(f'400 Bad Request')
            break
  if response.status_code == 500:
            print(f'500 Bad Request ')
            break
  response_json = response.json()
  for i in range(len(response_json['reviews'])):
    this_review = response_json['reviews'][i]
    #price = this_business['price'] if this_business['price'] else 'UNKNOWN'
    new_row = { ADD HERE}
    review = review.append(new_row, ignore_index=True)
  #print(response_json['businesses'][0]['name'], response_json, response)
  #print(response)

SyntaxError: ignored

In [ ]:
url = 'https://api.yelp.com/v3/businesses/' + 'DGhWO1sUWydVeR5j5ZZaMw' + '/reviews'

search_limit = 50
url_params = {
                'limit': search_limit,
}
response = requests.get(url, headers=headers, params=url_params)
response_json = response.json()
response_json

## What we need now:
1. decide if we want reviews table and import that
2. figure out hours and neighborhood